<a href="https://colab.research.google.com/github/MadgeneM/DDS8555/blob/main/MoiseMDDS8555_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Title: Assignment 1
# Author: Madgene Moise
# Date: Sunday, May 18, 2025

In [ ]:
# Load the IRIS dataset in Python
# Import pandas, sklearn, and numpy libraries

import pandas as pd
import seaborn as sns
import numpy as np

from sklearn import datasets
iris = pd.DataFrame(datasets.load_iris().data)
iris.columns = datasets.load_iris().feature_names
iris['type'] = datasets.load_iris().target
iris['type'] = iris['type'].astype('object')
iris

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),type
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [ ]:
# Create a new feature by multiplying sepal length by sepal width and dividing by petal length * petal width.
iris['new']=(iris.iloc[:,0]*iris.iloc[:,1])/(iris.iloc[:,2]*iris.iloc[:,3])

In [ ]:
# Sample 80% of the data for a training set stratifying on 'type'.
# Split into train and test
# Changed iloc slicing to include all four features (0 to 3 inclusive)
from sklearn.model_selection import train_test_split as tts
X_train, X_test, y_train, y_test = tts(iris.iloc[:,0:4], iris.iloc[:,4], test_size=0.2, random_state=42, stratify=iris.iloc[:,4])

In [ ]:
# On the test set, evaluate the following two estimators for sepal width using ME (mean of the forecast error or mean error),
# MPE (mean percent error), MAPE (mean absolute percent error),
# MAE (mean absolute error), and MSE (mean squared error).
# Mean of petal length calculated only on the training data
# Mean of sepal length minus petal width calculated only on the training data

from sklearn.metrics import mean_squared_error
# Mean of petal length
est1 = np.mean(X_train['petal length (cm)'])
# Mean of sepal length - petal length
est2 = np.mean(X_train['sepal length (cm)'] - X_train['petal width (cm)'])

# Ground truth: sepal width from the test set
actual = X_test['sepal width (cm)'].values

# Predicted values (same constant repeated)
pred1 = np.full_like(actual, est1)
pred2 = np.full_like(actual, est2)

# Repeat the estimates to match the length of y_test
est1=[est1]*len(y_test)

est2=[est2]*len(y_test)

# Define a function to calculate the Mean Squared Error
def myf(actual, predicted):
  return mean_squared_error(actual, predicted)

#Evaluate the estimates
result1 = myf(X_test['sepal width (cm)'],est1)

result2 = myf(X_test['sepal width (cm)'],est2)

print("MSE for est1:", result1)
print("MSE for est2:", result2)


MSE for est1: 0.6018333333333334
MSE for est2: 2.5258333333333334


In [ ]:
# Evaluate metrics
def evaluation_metrics(actual, predicted):
  error = actual - predicted
  percent_error = error / actual * 100
  abs_percent_error = np.abs(percent_error)

  return {
     'ME': np.mean(error),                         # Mean Error
     'MPE': np.mean(percent_error),                # Mean Percent Error
     'MAPE': np.mean(abs_percent_error),           # Mean Absolute Percent Error
     'MAE': np.mean(np.abs(error)),                # Mean Absolute Error
     'MSE': np.mean(error ** 2)                    # Mean Squared Error
    }

# Evaluate both estimators
results_est1 = evaluation_metrics(actual, pred1)
results_est2 = evaluation_metrics(actual, pred2)

# Display the results
print("Estimator 1 (mean of petal length) evaluation:")
for metric, value in results_est1.items():
    print(f"{metric}: {value:.4f}")

print("\nEstimator 2 (mean of sepal length - petal width) evaluation:")
for metric, value in results_est2.items():
    print(f"{metric}: {value:.4f}")


Estimator 1 (mean of petal length) evaluation:
ME: -0.6767
MPE: -23.7499
MAPE: 24.1859
MAE: 0.6940
MSE: 0.6018

Estimator 2 (mean of sepal length - petal width) evaluation:
ME: -1.5433
MPE: -52.1981
MAPE: 52.1981
MAE: 1.5433
MSE: 2.5258


In [ ]:
# On the test set, evaluate the two classifiers (built on the training set) below for 'type' using accuracy, precision, recall, and the F1 score.
# Up to 1st quantile sepal length = type 0, >1st up to 2nd quantile = type 1, >2d quantile = type 2.
# Up to 2d quantile of sepal length = type 0, >2nd up to 3d quantile = type 1, >3d quantile = type 2.

from numpy import percentile
from sklearn.metrics import classification_report as cr

# Classifier 1 is based on sepal length quantiles (Q1, Q2)
est3 = percentile(X_train['sepal length (cm)'], [25, 50])

y_hat=np.zeros(len(y_test))
y_hat[X_test['sepal length (cm)']>est3[0]]=1
y_hat[X_test['sepal length (cm)']>est3[1]]=2
y_hat=y_hat.astype('int')

print(cr(y_test.astype('int'),y_hat))

# Classifer 2 is based on quantiles (Q2, Q3)
est4 = percentile(X_train['sepal length (cm)'], [50,75])

y_hat2=np.zeros(len(y_test))
y_hat2[X_test['sepal length (cm)']>est4[0]]=1
y_hat2[X_test['sepal length (cm)']>est4[1]]=2
y_hat2=y_hat2.astype('int')

print(cr(y_test.astype('int'),y_hat2))

              precision    recall  f1-score   support

           0       0.71      0.50      0.59        10
           1       0.33      0.20      0.25        10
           2       0.59      1.00      0.74        10

    accuracy                           0.57        30
   macro avg       0.55      0.57      0.53        30
weighted avg       0.55      0.57      0.53        30

              precision    recall  f1-score   support

           0       0.69      0.90      0.78        10
           1       0.38      0.30      0.33        10
           2       0.67      0.60      0.63        10

    accuracy                           0.60        30
   macro avg       0.58      0.60      0.58        30
weighted avg       0.58      0.60      0.58        30

